In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [2]:
from fastai.text import *
from sebastian.eval import eval_ner

/root/anaconda3/envs/pytorch4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda3/envs/pytorch4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda3/envs/pytorch4/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/root/anaconda3/envs/pytorch4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda3/envs/pytorch4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may

In [3]:
from model.data_utils import minibatches, pad_sequences, get_chunks
from model.config import Config
from model.data_utils import CoNLLDataset
from torch.utils.data.sampler import SequentialSampler, RandomSampler, BatchSampler

In [4]:
config = Config()

In [5]:
train = CoNLLDataset(config.filename_train, config.processing_word,
                         config.processing_tag, config.max_iter)
val = CoNLLDataset(config.filename_dev, config.processing_word,
                         config.processing_tag, config.max_iter)
test = CoNLLDataset(config.filename_test, config.processing_word,
                         config.processing_tag, config.max_iter)

In [6]:
class Minibatch(object):
    def __init__(self, data, minibatch_size):
        self.data = data
        self.minibatch_size = minibatch_size
        self.length = None
        
    def __iter__(self):
        x_batch, y_batch = [], []
        for (x, y) in self.data:
            if len(x_batch) == self.minibatch_size:
                char_ids, word_ids = zip(*x_batch)
                word_ids, sequence_lengths = pad_sequences(word_ids, 0)
                char_ids, word_lengths = pad_sequences(char_ids, pad_tok=0,
                    nlevels=2)
                lbl_ids, lbl_lengths = pad_sequences(y_batch, 0)
                word_ids = np.concatenate([np.array(word_ids)[:,:,np.newaxis], np.array(char_ids)], axis=-1)
                yield T(word_ids), T(lbl_ids).view(-1)
                x_batch, y_batch = [], []

            if type(x[0]) == tuple:
                x = zip(*x)
            x_batch += [x]
            y_batch += [y]

        if len(x_batch) != 0:
            char_ids, word_ids = zip(*x_batch)
            word_ids, sequence_lengths = pad_sequences(word_ids, 0)
            char_ids, word_lengths = pad_sequences(char_ids, pad_tok=0,
                nlevels=2)
            lbl_ids, lbl_lengths = pad_sequences(y_batch, 0)
            word_ids = np.concatenate([np.array(word_ids)[:,:,np.newaxis], np.array(char_ids)], axis=-1)
            yield T(word_ids), T(lbl_ids).view(-1)
    
    def __len__(self):
        if self.length == None:
            self.length = 0
            for _ in self:
                self.length += 1
        return self.length

In [7]:
dir_path = '/home/'
trn_dl = Minibatch(train, 20)
val_dl = Minibatch(val, 20)
test_dl = Minibatch(test, 20)
md = ModelData(dir_path, trn_dl, val_dl, test_dl)

In [17]:
class NER_model(nn.Module):
    def __init__(self, config):
        super(NER_model, self).__init__()
        self.config = config
        self.idx_to_tag = {idx: tag for tag, idx in
                           self.config.vocab_tags.items()}
        self.get_word_embeddings()
        self.get_logits()
        
    def get_word_embeddings(self):
        # get word embeding
        _word_embedding = V(self.config.embeddings, requires_grad=True)
        self.word_embedding = nn.Embedding.from_pretrained(_word_embedding)
        
        # get char embedding
        self._char_embedding = nn.Embedding(self.config.nchars, self.config.dim_char)
        self.char_embedding = nn.LSTM(input_size=self.config.dim_char, hidden_size=self.config.hidden_size_char,
                                     num_layers=1, batch_first=True, # not sure here whether batch is first
                                     bidirectional=True)
        
    def get_logits(self):
        self.rnn = nn.LSTM(input_size=self.config.dim_word+self.config.dim_char*2,
                          hidden_size=self.config.hidden_size_lstm,
                          num_layers=1, batch_first=True, # not sure whether batch is first
                          bidirectional=True)
        self.dropout = nn.Dropout(self.config.dropout if self.train else 0)
        self.linear = nn.Linear(self.config.hidden_size_lstm*2, self.config.ntags)
        
    def forward(self, input):
        char = input[:, :, 1:]
        words = input[:, :, 0]
        bs, sl, _ = char.size()
        char = char.view(-1, _)
        _char_embedding = self._char_embedding(char)
        
        char_embedding, (h_n, cell_n) = self.char_embedding(_char_embedding)
        char_embedding = h_n.view(bs, sl, -1)
        word_embedding = self.word_embedding(words)

        # concat word embeddings and char embeddings
        word_embedding = torch.cat([word_embedding, char_embedding], dim=-1)
        word_embedding_dp = self.dropout(word_embedding)
        
        out, (n_h, n_cell) = self.rnn(word_embedding_dp)
        out_dp = self.dropout(out)
        out = self.linear(out_dp)
        return out.view(out.size(0)*out.size(1), out.size(-1))

In [18]:
ner_model = NER_model(config).cuda()

In [19]:
opt = optim.Adam(ner_model.parameters(), 1e-3)
loss_func = F.cross_entropy

In [20]:
##### rewrite RNN Learner #####
'''rewrite load_encoder to load the encoding modules'''
class RNN_Learner_bidir(Learner):
    def __init__(self, data, models, **kwargs):
        super().__init__(data, models, **kwargs)

    def _get_crit(self, data): return F.cross_entropy
    def fit(self, *args, **kwargs): return super().fit(*args, **kwargs, seq_first=True)

    def save_encoder(self, name): save_model(self.model[0], self.get_model_path(name))
    def load_encoder(self, name_fw, name_bw): 
        load_model(self.model.rnn_enc_fw, self.get_model_path(name_fw))
        load_model(self.model.rnn_enc_bw, self.get_model_path(name_bw))
##### end #####


##### rewrite textmodel #####
'''get layer groups'''
class TextModel_bidir(BasicModel):
    def get_layer_groups(self):
        return [(self.model._char_embedding),(self.model.char_embedding), 
                (self.model.word_embedding),(self.model.rnn),(self.model.linear)]

opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
learn = RNN_Learner_bidir(md, TextModel_bidir(to_gpu(ner_model)), opt_fn=opt_fn)

In [21]:
learn.fit(0.001, 2, metrics=[accuracy])

epoch      trn_loss   val_loss   accuracy                    
    0      0.116252   0.081484   0.976021  
    1      0.081436   0.055951   0.983435                     



[0.05595079365943407, 0.9834345496497173]

In [30]:
eval_ner(learn, id2tag, is_test=True)

Test f1 measure overall: 0.8294657964708226
{'precision-MISC': 0.6813186813186812, 'recall-MISC': 0.6182336182336181, 'f1-measure-MISC': 0.6482449589245207, 'precision-LOC': 0.9042647712127944, 'recall-LOC': 0.9320054945054945, 'f1-measure-LOC': 0.9179255918827008, 'precision-PER': 0.8096371207614516, 'recall-PER': 0.8437693738375698, 'f1-measure-PER': 0.8263509411049896, 'precision-ORG': 0.7203757225433526, 'recall-ORG': 0.6002408187838651, 'f1-measure-ORG': 0.6548440065680949, 'precision-overall': 0.8365431496830814, 'recall-overall': 0.8225071907957814, 'f1-measure-overall': 0.8294657964708226}
Test token-level accuracy of NER model: 0.9787.


In [27]:
id2tag = [o for i,o in ner_model.idx_to_tag.items()]

In [26]:
id2tag

{'0': 'I-LOC',
 '1': 'B-MISC',
 '2': 'B-LOC',
 '3': 'I-PER',
 '4': 'O',
 '5': 'I-ORG',
 '6': 'I-MISC',
 '7': 'B-ORG'}

In [28]:
id2tag

['I-LOC', 'B-MISC', 'B-LOC', 'I-PER', 'O', 'I-ORG', 'I-MISC', 'B-ORG']